In [1]:
import torch
import numpy as np

In [2]:
sentence = ("If you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))
char_to_index = {c: i for i, c in enumerate(char_set)}

In [4]:
char_size = len(char_set)
hidden_size = len(char_set)
sequence_length = 10
learning_rate = 0.1

In [5]:
x_data = []
y_data = []
for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, '->', y_str)

  x_data.append([char_to_index[c] for c in x_str])
  y_data.append([char_to_index[c] for c in y_str])
x_one_hot = [np.eye(char_size)[i] for i in x_data]

0 If you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [6]:
class RNN2(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers=1):
    super(RNN2, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x
model = RNN2(char_size, hidden_size, 2)

In [7]:
x_one_hot = np.array(x_one_hot)

In [19]:
X = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [21]:
# start training
for i in range(1000):
  optimizer.zero_grad()
  outputs = model(X)
  print(outputs.shape)
  print(outputs.view(-1, char_size).shape)
  break
  loss = criterion(outputs.view(-1, char_size), y.view(-1))
  loss.backward()
  optimizer.step()

  results = outputs.argmax(dim=2)
  predict_str = ""
  for j, result in enumerate(results):
    if j == 0:
      predict_str += ''.join([char_set[i] for i in result])
    else:
      predict_str += char_set[result[-1]]
  print(predict_str)

torch.Size([170, 10, 26])
torch.Size([1700, 26])


In [11]:
x_data2 = "I"
x_data2 = [np.eye(char_size)[char_to_index[x]] for x in x_data2]
x_data2

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1.])]

In [12]:
X = torch.FloatTensor(x_data2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [13]:
ret = ""
pred = model(X.view(1, 1, -1)).argmax(dim=2)
for i in range(1):
  ret += char_set[pred[0,i].item()]
print(ret)

f
